In [1]:
import requests
import bs4
import pandas
import os
import time

In [2]:
# 요청할 페이지의 주소
site = 'https://openapi.naver.com/v1/search/news.xml'
# 헤더 정보
client_Id = 'HCEMLBusbJrqI4Fv2tYf'
client_Secret = 'j6W3i_fYiA'
# 파라미터
query = '역삼역맛집'
start = 1
display = 10

In [3]:
header_dict = {
    'X-Naver-Client-Id' : client_Id,
    'X-Naver-Client-Secret' : client_Secret
}

param_dict = {
    'query' : query,
    'start' : 1,
    'display' : 10
}

In [4]:
while True :
    time.sleep(1)
    
    response = requests.get(site, headers=header_dict, params=param_dict)

    soup = bs4.BeautifulSoup(response.text, 'lxml')

    channel_tag = soup.find('channel')

    # 전체 검색 결과의 개수
    total_tag = channel_tag.find('total')
    total = total_tag.text

    start_tag = channel_tag.find('start')
    display_tag = channel_tag.find('display')

    start = start_tag.text
    display = display_tag.text

    print(f'total : {total}, start : {start}, display : {display}')

    # 항목 태그들을 가져온다.
    item_list = channel_tag.find_all('item')
    # 항목 데이터들을 가져온다.
    for item_tag in item_list :
        print(item_tag)
        title_tag = item_tag.find('title')
        originallink_tag = item_tag.find('originallink')
        link_tag = item_tag.find('link')
        description_tag = item_tag.find('description')
        pubDate_tag = item_tag.find('pubdate')
        
        df1 = pandas.DataFrame([
            [title_tag.text, originallink_tag.text,
             link_tag.text, description_tag.text,
             pubDate_tag.text]
        ])
        
        if os.path.exists('naver_api1.csv') == False :
            df1.columns = ['title', 'originallink', 'link',
                           'description', 'pubDate']
            df1.to_csv('naver_api1.csv', index=False, encoding='utf-8-sig')
        else :
            df1.to_csv('naver_api1.csv', index=False, encoding='utf-8-sig', mode='a', header=False)
        
    # 다음에 가져올 것이 있는지 검사
    now_cnt = int(start) + int(display) - 1
    if now_cnt < int(total) :
        param_dict['start'] = int(start) + 10
    else :
        break
        
#     if param_dict['start'] > 30 :
#         break
        
print('수집완료')

total : 363, start : 1, display : 10
<item><title>강남&lt;b&gt;역 맛집&lt;/b&gt; ‘봉우화로’, 신년모임 장소 찾는다면 주목</title><originallink>http://www.ihalla.com/read.php3?aid=1579767611672848348</originallink><link/>http://www.ihalla.com/read.php3?aid=1579767611672848348<description>수준 높은 음식과 차별화된 서비스만 제공하기 위해 노력할 것”이라고 전했다. 단체 예약 및 기타 자세한 사항은 서울 강남구 &lt;b&gt;역삼&lt;/b&gt;동 강남&lt;b&gt;역 맛집&lt;/b&gt; ‘봉우화로’로 문의하면 된다.</description><pubdate>Thu, 23 Jan 2020 17:22:00 +0900</pubdate></item>


PermissionError: [Errno 13] Permission denied: 'naver_api1.csv'